In [38]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import librosa
import numpy as np
import librosa.display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from server.database_wrapper import PostgresqlWrapper
from server.utils import Util
from server.feature_extractor import FeatureAggregator
import xgboost as xgb
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from numpy import binary_repr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Моя портянка

In [310]:
# Код для проверки работоспособности модели:
# Берем разные рандонстэйты, делим выборку, выводим средний classification_report

def check_model(model, X, y, n, genres):
    average = np.zeros((len(genres) + 1, 4), dtype = float)
    skf = StratifiedKFold(n_splits=n)
    skf.get_n_splits(X, y)
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i, stratify=y)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_scaled_train = scaler.transform(X_train)
        X_scaled_test = scaler.transform(X_test)
        model.fit(X_scaled_train, y_train)
        df = parse_class_report(classification_report(y_test, model.predict(X_scaled_test), target_names=genres))
        average += df.values
    df.iloc[:,:] = np.round(average / n, 3)
    return df

def parse_class_report(class_rep_str):
    lines = class_rep_str.split('\n')
    list_splitted = list()
    for line in lines:
        splitted_line = np.array(line.split(' '))
        mask_non_empty = np.array([word != '' for word in line.split(' ')])
        res = splitted_line[mask_non_empty]
        if len(res) != 0:
            list_splitted.append(np.array(res, dtype = object))

    first = list(list_splitted[0])
    first.insert(0, 'score')
    list_splitted[0] = np.array(first, dtype = object)
    list_splitted[-1] = list_splitted[-1][2:]
    list_splitted[-1][0] = 'avg / total'
    
    df = pd.DataFrame(list_splitted)
    df.columns = list(df.iloc[0])
    return df.iloc[1:].set_index('score').convert_objects(convert_numeric=True)

In [237]:
data = pd.DataFrame.from_csv("extracted_data_new.csv", encoding="utf-8").values
X = np.array(data[:, :-1], dtype = float)
genre_list = data[:, -1]
y = np.arange(10).repeat(100)

In [48]:
y = np.arange(10).repeat(100)

In [52]:
rfc = RandomForestClassifier()
svc = SVC()
skf = StratifiedKFold(n_splits=3)

In [53]:
param_grid = { 
    'n_estimators': [ 70, 150, 370],
    'max_features': ['log2'],
    'max_depth' : [10,15, 20],
    'criterion' :['gini']
}

grid_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=skf, n_jobs = -1, verbose=True)
grid_svc = GridSearchCV(svc, {"C": np.logspace(0,2,num=20), "kernel": ["poly", "rbf", "sigmoid"]}, n_jobs = -1, verbose=True, cv = skf)
eclf = VotingClassifier(estimators=[('svc', grid_svc), ('rfc', grid_rfc)], voting='hard')

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9, stratify=y)

In [54]:
# Normalization
scaler = StandardScaler()
scaler.fit(X)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [55]:
eclf.fit(X_scaled_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    2.1s


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    2.8s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    5.3s finished


VotingClassifier(estimators=[('svc', GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=F...pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [59]:
must_have_genres = ['classical', 'metal', 'hiphop', 'rock', 'pop', 'jazz']
include_genres = ['country', 'blues', 'disco', 'reggae']

In [311]:
models_list = list()
for i in tqdm(range(16)):
    mask_subset = np.array(list(binary_repr(i,width=4))) == '1'
    genres_to_fit = np.array(must_have_genres + list(np.array(include_genres)[mask_subset]) )
    mask = np.zeros_like(genre_list, dtype=bool)
    for genre in genres_to_fit:
        mask = mask | (genre_list == genre)
    genres_to_fit = np.unique(genre_list[mask], return_index=True)
    genres_to_fit = genres_to_fit[0][np.argsort(genres_to_fit[1])]
    X = np.array(data_to_save[:, :-1], dtype = float)     
    X = X[mask]
    y = np.arange(len(genres_to_fit)).repeat(100)
    
    models_list.append(check_model(eclf, X, y, 10, genres_to_fit))

  0%|          | 0/16 [00:00<?, ?it/s]

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.5s finished


Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.6s finished


...

[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    7.3s finished


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.3s finished


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    7.2s finished
100%|██████████| 16/16 [25:57<00:00, 110.10s/it]


In [312]:
max_6 = 0.
max_7 = 0.
max_8 = 0.
max_9 = 0.
max_10 = 0.

model_6 = pd.DataFrame()
model_7 = pd.DataFrame()
model_8 = pd.DataFrame()
model_9 = pd.DataFrame()
model_10 = pd.DataFrame()
for model in models_list:
    if model.values.shape[0] - 1 == 6 and max_6 < model.values[-1,0]:
        max_6 = model.values[-1,0]
        model_6 = model
    if model.values.shape[0] - 1 == 7 and max_7 < model.values[-1,0]:
        max_7 = model.values[-1,0]
        model_7 = model
    if model.values.shape[0] - 1 == 8 and max_8 < model.values[-1,0]:
        max_8 = model.values[-1,0]
        model_8 = model
    if model.values.shape[0] - 1 == 9 and max_9 < model.values[-1,0]:
        max_9 = model.values[-1,0]
        model_9 = model
    if model.values.shape[0] - 1 == 10 and max_10 < model.values[-1,0]:
        max_10 = model.values[-1,0]
        model_10 = model

    

In [313]:
model_6

,precision,recall,f1-score,support
score,,,,
classical,0.873,0.940,0.906,30.0
metal,0.783,0.896,0.834,30.0
hiphop,0.741,0.838,0.786,30.0
pop,0.799,0.790,0.791,30.0
rock,0.765,0.614,0.678,30.0
jazz,0.928,0.774,0.843,30.0
avg / total,0.814,0.808,0.806,180.0


In [314]:
model_7

,precision,recall,f1-score,support
score,,,,
classical,0.880,0.933,0.905,30.0
metal,0.783,0.877,0.826,30.0
hiphop,0.700,0.791,0.742,30.0
pop,0.748,0.799,0.771,30.0
rock,0.691,0.608,0.643,30.0
reggae,0.762,0.637,0.692,30.0
jazz,0.902,0.785,0.836,30.0
avg / total,0.781,0.778,0.771,210.0


In [315]:
model_8

,precision,recall,f1-score,support
score,,,,
classical,0.878,0.934,0.905,30.0
metal,0.740,0.883,0.804,30.0
blues,0.656,0.813,0.725,30.0
hiphop,0.756,0.810,0.779,30.0
pop,0.758,0.779,0.765,30.0
rock,0.555,0.433,0.485,30.0
country,0.769,0.541,0.628,30.0
jazz,0.881,0.727,0.791,30.0
avg / total,0.748,0.739,0.736,240.0


In [316]:
model_9

,precision,recall,f1-score,support
score,,,,
classical,0.865,0.947,0.902,30.0
metal,0.737,0.859,0.791,30.0
blues,0.657,0.807,0.722,30.0
hiphop,0.684,0.754,0.712,30.0
pop,0.732,0.797,0.761,30.0
rock,0.525,0.445,0.482,30.0
country,0.727,0.547,0.621,30.0
reggae,0.710,0.600,0.650,30.0
jazz,0.895,0.702,0.784,30.0


In [317]:
model_10

,precision,recall,f1-score,support
score,,,,
classical,0.860,0.936,0.895,30.0
metal,0.708,0.877,0.783,30.0
blues,0.715,0.784,0.743,30.0
hiphop,0.613,0.694,0.649,30.0
disco,0.523,0.631,0.569,30.0
pop,0.706,0.749,0.725,30.0
rock,0.450,0.330,0.377,30.0
country,0.714,0.615,0.658,30.0
reggae,0.725,0.546,0.619,30.0


In [ ]:
aveg = 0.
for i in range(10):
    aveg += np.mean(eclf.predict(X_scaled_test) == y_test) / 30
aveg / 8

In [ ]:
np.mean(eclf.predict(X_scaled_test) == y_test)

## Портянка Стёпы

In [ ]:
data = list()
genre_list = list()

In [2]:
genres=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock', 'country', 'reggae', 'jazz']

In [ ]:
def getData(where_to, genre_list, genre, range_tuple):
    for i in tqdm(range(range_tuple)):
        if i < 10:
            path = "../../" + genre + "/" + genre + ".0000" + str(i) + ".au"        
        else:
            path = "../../" + genre + "/" + genre + ".000" + str(i) + ".au"
        
        song = librosa.load(path)
        where_to.append(song[0])
        genre_list.append(genre)

In [ ]:
for genre in genres:
    getData(data, genre_list, genre, 100)

In [5]:
def parallel_get_data(genre, range_tuple=100):
    data = list()
    for i in range(range_tuple):
        if i < 10:
            path = "../../" + genre + "/" + genre + ".0000" + str(i) + ".au"        
        else:
            path = "../../" + genre + "/" + genre + ".000" + str(i) + ".au"
        song = librosa.load(path)
        data.append(song[0])
        if i%30 == 0: print("Got {0} songs for genre {1}".format(i, genre))
    return (data, genre)

In [6]:
from concurrent.futures import ThreadPoolExecutor

In [7]:
with ThreadPoolExecutor(len(genres)) as pool:
    results = pool.map(parallel_get_data, genres)

Got 0 songs for genre jazz
Got 0 songs for genre rock
Got 0 songs for genre metal
Got 0 songs for genre country
Got 0 songs for genre reggae
Got 0 songs for genre pop
Got 0 songs for genre classicalGot 0 songs for genre hiphop

Got 0 songs for genre disco
Got 0 songs for genre blues
Got 30 songs for genre hiphop
Got 30 songs for genre classical
Got 30 songs for genre metal
Got 30 songs for genre disco
Got 30 songs for genre jazzGot 30 songs for genre rock

Got 30 songs for genre pop
Got 30 songs for genre country
Got 30 songs for genre reggae
Got 30 songs for genre blues
Got 60 songs for genre classical
Got 60 songs for genre hiphop
Got 60 songs for genre metal
Got 60 songs for genre rock
Got 60 songs for genre jazz
Got 60 songs for genre pop
Got 60 songs for genre blues
Got 60 songs for genre reggae
Got 60 songs for genre disco
Got 60 songs for genre country
Got 90 songs for genre classical
Got 90 songs for genre hiphop
Got 90 songs for genre metal
Got 90 songs for genre jazz
Got 90 s

In [8]:
results = list(results)

In [27]:
len(results[5][0])

100

In [32]:
music_list = list()
genre_list = list()
for genre_data in np.array(results): 
    music_list += genre_data[0]
    genre_list += list(np.array([genre_data[1]]).repeat(100))

In [39]:
exctractor = FeatureAggregator(music_list, parallel=True)

In [40]:
data = exctractor.get_features()

Got mfcc for 0 songs
Got zero_cross_rate for 0 songsGot centroid data for 0 songs
Got rhythm data for 0 songs

Got rmse data for 0 songs
Got zero_cross_rate for 100 songs
Got zero_cross_rate for 200 songs
Got zero_cross_rate for 300 songs
Got mfcc for 100 songs
Got zero_cross_rate for 400 songs
Got zero_cross_rate for 500 songs
Got zero_cross_rate for 600 songs
Got centroid data for 100 songs
Got zero_cross_rate for 700 songs
Got mfcc for 200 songs
Got rhythm data for 100 songs
Got zero_cross_rate for 800 songs
Got zero_cross_rate for 900 songs
Done  <class 'server.feature_extractor.ZeroCrossing'>
Got mfcc for 300 songs
Got centroid data for 200 songs
Got rhythm data for 200 songs
Got mfcc for 400 songs
Got centroid data for 300 songs
Got mfcc for 500 songs
Got rmse data for 100 songs
Got rhythm data for 300 songs
Got mfcc for 600 songs
Got centroid data for 400 songs
Got mfcc for 700 songs
Got rhythm data for 400 songs
Got mfcc for 800 songs
Got centroid data for 500 songs
Got mfcc fo

In [44]:
data_to_save = np.hstack((data, np.array(genre_list).reshape(-1,1)))

In [308]:
data_to_save.shape

(1000, 63)

In [46]:
pd.DataFrame(data_to_save).to_csv("extracted_data_new.csv", encoding="utf-8")

In [47]:
X = data

In [57]:
print(classification_report(y_test, eclf.predict(X_scaled_test), target_names=genres))

             precision    recall  f1-score   support

  classical       0.88      0.93      0.90        30
      metal       0.78      0.83      0.81        30
      blues       0.67      0.87      0.75        30
     hiphop       0.66      0.63      0.64        30
      disco       0.51      0.60      0.55        30
        pop       0.88      0.93      0.90        30
       rock       0.48      0.33      0.39        30
    country       0.76      0.63      0.69        30
     reggae       0.58      0.70      0.64        30
       jazz       0.89      0.57      0.69        30

avg / total       0.71      0.70      0.70       300



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [69]:
import warnings
warnings.filterwarnings('ignore')

models_list = list()
for i in tqdm(range(16)):
    mask_subset = np.array(list(binary_repr(i,width=4))) == '1'
    genres_to_fit = np.array(must_have_genres + list(np.array(include_genres)[mask_subset]) )
    mask = np.zeros_like(genre_list, dtype=bool)
    for genre in genres_to_fit:
        mask = mask | (genre_list == genre)
    genres_to_fit = np.unique(np.array(genre_list)[mask], return_index=True)
    genres_to_fit = genres_to_fit[0][np.argsort(genres_to_fit[1])]
    X = np.array(data[:, :-1], dtype = float)     
    X = X[mask]
    y = np.arange(len(genres_to_fit)).repeat(100)
    
    models_list.append(check_model(eclf, X, y, 5, genres_to_fit))

  0%|          | 0/16 [00:00<?, ?it/s]

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.5s finished


Fitting 3 folds for each of 60 candidates, totalling 180 fits


/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `

...

[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   12.3s finished
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    3.4s


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    4.0s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   11.9s finished
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.

In [61]:
X.shape

(1000, 62)

In [62]:
genre_list

['classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'classical',
 'clas

In [75]:
import warnings
warnings.filterwarnings('ignore')

genres_to_fit = np.array(must_have_genres)
mask = np.zeros_like(genre_list, dtype=bool)
for genre in genres_to_fit:
    mask = mask | (genre_list == genre)
genres_to_fit = np.unique(np.array(genre_list)[mask], return_index=True)
genres_to_fit = genres_to_fit[0][np.argsort(genres_to_fit[1])]
X = np.array(data[:, :-1], dtype = float)     
X = X[mask]
y = np.arange(len(genres_to_fit)).repeat(100)

models_list.append(check_model(eclf, X, y, 5, genres_to_fit))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.3s finished


Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    0.9s finished


KeyboardInterrupt: 

In [238]:

genres_to_fit = np.array(must_have_genres)
mask = np.zeros_like(genre_list, dtype=bool)
for genre in genres_to_fit:
    mask = mask | (genre_list == genre)
genres_to_fit = np.unique(np.array(genre_list)[mask], return_index=True)
genres_to_fit = genres_to_fit[0][np.argsort(genres_to_fit[1])]
X = np.array(data[:, :-1], dtype = float)     
X = X[mask]
y = np.arange(len(genres_to_fit)).repeat(100)

In [304]:
a = list()
for rs in range(200):
    X_train_idx, X_test_idx, y_train_, y_test_ = train_test_split(np.arange(1000)[mask], y, test_size=0.2, random_state=rs, stratify=y)
    if 158 in X_test_idx:
        a.append(rs)
a

[0,
 2,
 11,
 17,
 20,
 33,
 35,
 40,
 47,
 53,
 54,
 57,
 60,
 71,
 73,
 75,
 85,
 86,
 87,
 99,
 100,
 107,
 108,
 110,
 112,
 121,
 122,
 129,
 142,
 153,
 157,
 158,
 168,
 173,
 174,
 176,
 178,
 180,
 185,
 188,
 192]

In [293]:
rs = 33

In [306]:
model = eclf

for rs in a:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=rs, stratify=y)
    X_train_idx, X_test_idx, y_train_, y_test_ = train_test_split(np.arange(1000)[mask], y, test_size=0.25, random_state=rs, stratify=y)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_scaled_train = scaler.transform(X_train)
    X_scaled_test = scaler.transform(X_test)
    model.fit(X_scaled_train, y_train)
    if 158 in list(X_test_idx[(eclf.predict(X_scaled_test) == 4) & (y_test != 4) ]):
        print(rs, classification_report(y_test, model.predict(X_scaled_test), target_names=genres_to_fit))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.6s finished


0              precision    recall  f1-score   support

  classical       0.96      0.96      0.96        25
      metal       0.77      0.92      0.84        25
     hiphop       0.87      0.80      0.83        25
        pop       0.74      0.92      0.82        25
       rock       0.79      0.60      0.68        25
       jazz       0.95      0.84      0.89        25

avg / total       0.85      0.84      0.84       150

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.2s finished


Fitting 3 folds for each of 9 candidates, totalling 27 fits


...

[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    8.1s finished


185              precision    recall  f1-score   support

  classical       0.96      0.92      0.94        25
      metal       0.85      0.88      0.86        25
     hiphop       0.75      0.84      0.79        25
        pop       0.67      0.80      0.73        25
       rock       0.68      0.60      0.64        25
       jazz       0.95      0.76      0.84        25

avg / total       0.81      0.80      0.80       150

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    8.2s finished


Fitting 3 folds for each of 60 candidates, totalling 180 fits
Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    6.2s finished


In [302]:
# Как классифицируются роковые песни?
pd.DataFrame(
[list(genres_to_fit[eclf.predict(X_scaled_test)[y_test == 4]][eclf.predict(X_scaled_test)[y_test == 4] != 4]),
list(X_test_idx[y_test == 4][eclf.predict(X_scaled_test)[y_test == 4] != 4])])

IndexError: boolean index did not match indexed array along dimension 0; dimension is 150 but corresponding boolean dimension is 120

In [297]:
# Что классификатор считает роком, что в датасете роком не является?
pd.DataFrame(
[list(genres_to_fit[y_test[(eclf.predict(X_scaled_test) == 4) & (y_test != 4) ]]),
list(X_test_idx[(eclf.predict(X_scaled_test) == 4) & (y_test != 4) ])])

,0,1,2,3,4,5
0,pop,pop,hiphop,pop,jazz,jazz
1,517,572,313,581,973,983


In [292]:
X_scaled_test.shape

(150, 62)

In [231]:
extractor = FeatureAggregator([librosa.load("../../Алла Пугачева – Миллион алых роз.mp3")[0]], parallel=False)

In [232]:
eclf.predict(extractor.get_features())

Got mfcc for 0 songs
Got zero_cross_rate for 0 songs
Got centroid data for 0 songs
Got rhythm data for 0 songs
Got rmse data for 0 songs


ValueError: Found array with dim 3. Estimator expected <= 2.

In [245]:
data=extractor.get_features()

Got mfcc for 0 songs
Got zero_cross_rate for 0 songs
Got centroid data for 0 songs
Got rhythm data for 0 songs
Got rmse data for 0 songs


In [246]:
eclf.predict(data[0])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [248]:
data[0]

array([[-8.52029513e+01,  1.10406762e+02,  7.31618635e+00,
         2.99264821e+01, -1.54672557e+01, -3.63413301e+00,
        -1.77409160e+00, -1.80134621e+00, -8.79118850e+00,
        -5.43732462e+00, -1.90252566e+01,  2.75554558e+00,
        -9.20872276e+00, -5.72896265e+00, -1.06757471e+01,
        -2.42547860e+00, -8.69946552e+00, -8.41895202e+00,
        -7.66319548e+00,  2.13474110e-01,  6.10169725e+42,
         7.61718750e-02,  8.67945334e-02,  5.82684352e-02,
         1.31835938e-02,  5.86425781e-01,  1.77693243e+03,
         1.94018482e+03,  9.11987043e+02,  3.88102206e+02,
         6.87217604e+03,  3.94057617e+03,  4.12420165e+03,
         1.84038299e+03,  4.41430664e+02,  9.63610840e+03,
         1.61499023e+02,  4.70672531e-01,  7.61151622e-02,
         8.01334569e-02,  1.74062884e-01,  2.07763486e-01,
        -2.83298949e-15,  1.00000000e+00,  2.83236381e-01,
         2.56098279e-01,  2.11794417e-01,  1.94971599e-01,
         2.81118507e-01,  3.58965351e-01,  3.21917821e-0